### Laboratorio 8 - Servidores
Stefano Aragoni, Carol Arevalo, Luis Diego Santos

---------------

Suponga que usted está a cargo de definir la arquitectura a usar en el lanzamiento de su próxima aplicación web: C3 (sistema de contabilidad de la carreta contadora). La junta directiva le ha solicitado que encuentre el mejor servicio de hosting para el proyecto. Después de una investigación gigante, usted concluye que las mejores opciones se reducen a las siguientes dos:

1. **Proveedor 1 - Mountain Mega Computing:** Tienen una infraestructura de servidor único, con mucha potencia de procesamiento. Ellos se enorgullecen al indicar que *su servidor Enterprise puede atender hasta 100 solicitudes por segundo*.

2. **Proveedor 2 - Pizzita computing:** Tienen una infraestructura de múltiples servidores (en nube). Cada servidor es medianamente potente, y en su promoción indican que se paga únicamente la cantidad de servidores que su aplicación requiera. Luego de su análisis de esta oferta, usted infiere que cada servidor tiene a lo sumo una décima parte de la potencia del servidor promocionado por Mountain Mega Computing (*hasta 10 solicitudes por segundo*).


Las pruebas de estrés iniciales, y las proyecciones calculadas para los primeros dos años luego del lanzamiento, indican que <font color=red>su aplicación jamás excederá los 2,400 solicitudes por minuto</font>. Una auditoría y análisis de benchmark a sistemas similares al suyo, indican que <font color=orange>las solicitudes deberían llegar como un proceso de Poisson</font>, y que <font color=green>el tiempo de servicio de cada solicitud (sin importar la arquitectura de servidor usada) es modelado adecuadamente por una variable aleatoria exponencial</font>.


Mañana tiene que presentar su decisión final a la junta directiva del proyecto. Como no tiene tiempo para hacer una investigación a detalle con los clientes de cada proveedor, decide creer en su promoción y hacer una simulación para concluir cuál será la mejor opción.

--------------

#### Simulación - Tasks

1. Modele, simule y analice el comportamiento de ambos sistemas durante una hora de ejecución de C3, y para cada sistema responda

In [10]:
import simpy
import numpy as np

# Parámetros de la simulación
LAMBDA = 2400/60  # solicitudes por segundo
MU = 100  # solicitudes por segundo que puede atender el servidor de Mountain Mega Computing
TIME = 3600  # 1 hora

def source(env, number, lambda_, server, service_time):
    """Genera solicitudes aleatoriamente"""
    for i in range(int(number)):
        c = request(env, 'Request%02d' % i, server, service_time)
        env.process(c)
        t = np.random.exponential(1.0/lambda_)
        yield env.timeout(t)

def request(env, name, server, service_time):
    """Una solicitud llega al servidor para ser atendida"""
    arrive = env.now
    with server.request() as req:
        results = yield req

        wait = env.now - arrive
        yield env.timeout(service_time)
        total_wait.append(wait)
        total_service.append(service_time)
        total_served.append(1)

# Simulación para el Proveedor 1
env1 = simpy.Environment()
server1 = simpy.Resource(env1, capacity=1)
total_wait, total_service, total_served = [], [], []
env1.process(source(env1, TIME*LAMBDA, LAMBDA, server1, 1.0/MU))
env1.run(until=TIME)

# Recopilar estadísticas para Proveedor 1
total_requests_1 = sum(total_served)
total_wait_time_1 = sum(total_wait)
total_service_time_1 = sum(total_service)
idle_time_1 = TIME - total_service_time_1
average_queue_1 = len(total_wait) / TIME

print("\nProveedor 1 - Estadísticas:")
print(f"a. Solicitudes atendidas: {total_requests_1}")
print(f"b. Tiempo ocupado: {total_service_time_1}")
print(f"c. Tiempo desocupado: {idle_time_1}")
print(f"d. Tiempo total en cola: {total_wait_time_1}")
print(f"e. Tiempo promedio en cola: {total_wait_time_1 / total_requests_1}")
print(f"f. Promedio de solicitudes en cola por segundo: {average_queue_1}")
print(f"g. Momento de la salida de la última solicitud: {TIME}")

# Simulación para el Proveedor 2
env2 = simpy.Environment()
server2 = simpy.Resource(env2, capacity=10)
total_wait, total_service, total_served = [], [], []
env2.process(source(env2, TIME*LAMBDA, LAMBDA, server2, 10.0/MU)) # Decima parte de la potencia del servidor 1
env2.run(until=TIME)

# Recopilar estadísticas para Proveedor 2
total_requests_2 = sum(total_served)
total_wait_time_2 = sum(total_wait)
total_service_time_2 = sum(total_service)
idle_time_2 = TIME * 10 - total_service_time_2 # Para 10 servidores
average_queue_2 = len(total_wait) / TIME

print("\nProveedor 2 - Estadísticas:")
print(f"a. Solicitudes atendidas: {total_requests_2}")
print(f"b. Tiempo ocupado: {total_service_time_2}")
print(f"c. Tiempo desocupado: {idle_time_2}")
print(f"d. Tiempo total en cola: {total_wait_time_2}")
print(f"e. Tiempo promedio en cola: {total_wait_time_2 / total_requests_2}")
print(f"f. Promedio de solicitudes en cola por segundo: {average_queue_2}")
print(f"g. Momento de la salida de la última solicitud: {TIME}")



Proveedor 1 - Estadísticas:
a. Solicitudes atendidas: 143945
b. Tiempo ocupado: 1439.45
c. Tiempo desocupado: 2160.55
d. Tiempo total en cola: 475.9282526066765
e. Tiempo promedio en cola: 0.0033063201403777586
f. Promedio de solicitudes en cola por segundo: 39.984722222222224
g. Momento de la salida de la última solicitud: 3600

Proveedor 2 - Estadísticas:
a. Solicitudes atendidas: 144000
b. Tiempo ocupado: 14400.0
c. Tiempo desocupado: 21600.0
d. Tiempo total en cola: 15.959870971545136
e. Tiempo promedio en cola: 0.00011083243730239678
f. Promedio de solicitudes en cola por segundo: 40.0
g. Momento de la salida de la última solicitud: 3600


---------

2. Determine empíricamente cuántos servidores se necesitaría “alquilar” en Pizzita computing para asegurar que siempre habrá al menos un servidor disponible para atender una solicitud dada (en otras palabras, una solicitud nunca tiene que esperar en cola)

Para determinar empíricamente cuántos servidores se necesitan en Pizzita computing,  se realizarán múltiples simulaciones, cada una con un número incremental de servidores, hasta que lleguemos a un punto donde ninguna solicitud tiene que esperar en cola.

Este código inicializará una simulación con un servidor y la aumentará gradualmente hasta que no haya tiempo de espera. Al final, imprimirá cuántos servidores se necesitan para que ninguna solicitud espere en cola

In [11]:
# Parámetros de la simulación
LAMBDA = 2400/60  # solicitudes por segundo
MU = 100  # solicitudes por segundo que puede atender el servidor de Mountain Mega Computing
TIME = 3600  # 1 hora

def source(env, number, lambda_, server, service_time):
    """Genera solicitudes aleatoriamente"""
    for i in range(int(number)):
        c = request(env, 'Request%02d' % i, server, service_time)
        env.process(c)
        t = np.random.exponential(1.0/lambda_)
        yield env.timeout(t)

def request(env, name, server, service_time):
    """Una solicitud llega al servidor para ser atendida"""
    arrive = env.now
    with server.request() as req:
        results = yield req

        wait = env.now - arrive
        yield env.timeout(service_time)
        total_wait.append(wait)
        total_served.append(1)

num_servers = 1
while True:
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=num_servers)
    total_wait, total_served = [], []
    env.process(source(env, TIME*LAMBDA, LAMBDA, server, 10.0/MU)) # 10.0/MU para Pizzita
    env.run(until=TIME)

    total_requests = sum(total_served)
    total_wait_time = sum(total_wait)
    average_wait_time = total_wait_time / total_requests if total_requests != 0 else 0

    print(f"Con {num_servers} servidor(es), el tiempo promedio de espera es: {average_wait_time:.4f} segundos")

    if average_wait_time < 1e-4:  # Aproximado a cero
        print(f"\nSe requieren al menos {num_servers} servidores en Pizzita computing para asegurar que una solicitud nunca espera en cola.")
        break

    num_servers += 1


Con 1 servidor(es), el tiempo promedio de espera es: 1349.8000 segundos
Con 2 servidor(es), el tiempo promedio de espera es: 904.8892 segundos
Con 3 servidor(es), el tiempo promedio de espera es: 447.9486 segundos
Con 4 servidor(es), el tiempo promedio de espera es: 2.9021 segundos
Con 5 servidor(es), el tiempo promedio de espera es: 0.0294 segundos
Con 6 servidor(es), el tiempo promedio de espera es: 0.0077 segundos
Con 7 servidor(es), el tiempo promedio de espera es: 0.0026 segundos
Con 8 servidor(es), el tiempo promedio de espera es: 0.0010 segundos
Con 9 servidor(es), el tiempo promedio de espera es: 0.0003 segundos
Con 10 servidor(es), el tiempo promedio de espera es: 0.0001 segundos
Con 11 servidor(es), el tiempo promedio de espera es: 0.0000 segundos

Se requieren al menos 11 servidores en Pizzita computing para asegurar que una solicitud nunca espera en cola.


---------

3. Se espera que a partir del tercer año del lanzamiento de su aplicación, la cantidad de usuarios sufra un alza, y por tanto deberán atender como máximo 6000 solicitudes por minuto. Resuelva el inciso 1 y 2 para esta nueva configuración.

In [12]:
# Parámetros de la simulación
LAMBDA = 6000/60  # solicitudes por segundo con el nuevo valor
MU = 100  # solicitudes por segundo que puede atender el servidor de Mountain Mega Computing
TIME = 3600  # 1 hora

def source(env, number, lambda_, server, service_time):
    """Genera solicitudes aleatoriamente"""
    for i in range(int(number)):
        c = request(env, 'Request%02d' % i, server, service_time)
        env.process(c)
        t = np.random.exponential(1.0/lambda_)
        yield env.timeout(t)

def request(env, name, server, service_time):
    """Una solicitud llega al servidor para ser atendida"""
    arrive = env.now
    with server.request() as req:
        results = yield req

        wait = env.now - arrive
        yield env.timeout(service_time)
        total_wait.append(wait)
        total_service.append(service_time)
        total_served.append(1)

# Simulación para el Proveedor 1
env1 = simpy.Environment()
server1 = simpy.Resource(env1, capacity=1)
total_wait, total_service, total_served = [], [], []
env1.process(source(env1, TIME*LAMBDA, LAMBDA, server1, 1.0/MU))
env1.run(until=TIME)

# Recopilar estadísticas para Proveedor 1
total_requests_1 = sum(total_served)
total_wait_time_1 = sum(total_wait)
total_service_time_1 = sum(total_service)
idle_time_1 = TIME - total_service_time_1
average_queue_1 = len(total_wait) / TIME

print("\nProveedor 1 - Estadísticas:")
print(f"a. Solicitudes atendidas: {total_requests_1}")
print(f"b. Tiempo ocupado: {total_service_time_1}")
print(f"c. Tiempo desocupado: {idle_time_1}")
print(f"d. Tiempo total en cola: {total_wait_time_1}")
print(f"e. Tiempo promedio en cola: {total_wait_time_1 / total_requests_1}")
print(f"f. Promedio de solicitudes en cola por segundo: {average_queue_1}")
print(f"g. Momento de la salida de la última solicitud: {TIME}")

# Simulación para el Proveedor 2
env2 = simpy.Environment()
server2 = simpy.Resource(env2, capacity=10)
total_wait, total_service, total_served = [], [], []
env2.process(source(env2, TIME*LAMBDA, LAMBDA, server2, 10.0/MU)) # Nota el 10.0/MU aquí
env2.run(until=TIME)

# Recopilar estadísticas para Proveedor 2
total_requests_2 = sum(total_served)
total_wait_time_2 = sum(total_wait)
total_service_time_2 = sum(total_service)
idle_time_2 = TIME * 10 - total_service_time_2 # Para 10 servidores
average_queue_2 = len(total_wait) / TIME

print("\nProveedor 2 - Estadísticas:")
print(f"a. Solicitudes atendidas: {total_requests_2}")
print(f"b. Tiempo ocupado: {total_service_time_2}")
print(f"c. Tiempo desocupado: {idle_time_2}")
print(f"d. Tiempo total en cola: {total_wait_time_2}")
print(f"e. Tiempo promedio en cola: {total_wait_time_2 / total_requests_2}")
print(f"f. Promedio de solicitudes en cola por segundo: {average_queue_2}")
print(f"g. Momento de la salida de la última solicitud: {TIME}")



Proveedor 1 - Estadísticas:
a. Solicitudes atendidas: 359345
b. Tiempo ocupado: 3593.4500000000003
c. Tiempo desocupado: 6.549999999999727
d. Tiempo total en cola: 1492625.8756564877
e. Tiempo promedio en cola: 4.153740487989224
f. Promedio de solicitudes en cola por segundo: 99.81805555555556
g. Momento de la salida de la última solicitud: 3600

Proveedor 2 - Estadísticas:
a. Solicitudes atendidas: 359695
b. Tiempo ocupado: 35969.5
c. Tiempo desocupado: 30.5
d. Tiempo total en cola: 973032.8274250136
e. Tiempo promedio en cola: 2.7051608374456517
f. Promedio de solicitudes en cola por segundo: 99.91527777777777
g. Momento de la salida de la última solicitud: 3600


In [13]:
# Parámetros de la simulación
LAMBDA = 6000/60  # solicitudes por segundo con el nuevo valor
MU = 100  # solicitudes por segundo que puede atender el servidor de Mountain Mega Computing
TIME = 3600  # 1 hora

def source(env, number, lambda_, server, service_time):
    """Genera solicitudes aleatoriamente"""
    for i in range(int(number)):
        c = request(env, 'Request%02d' % i, server, service_time)
        env.process(c)
        t = np.random.exponential(1.0/lambda_)
        yield env.timeout(t)

def request(env, name, server, service_time):
    """Una solicitud llega al servidor para ser atendida"""
    arrive = env.now
    with server.request() as req:
        results = yield req

        wait = env.now - arrive
        yield env.timeout(service_time)
        total_wait.append(wait)
        total_served.append(1)

num_servers = 1
while True:
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=num_servers)
    total_wait, total_served = [], []
    env.process(source(env, TIME*LAMBDA, LAMBDA, server, 10.0/MU)) # 10.0/MU para Pizzita
    env.run(until=TIME)

    total_requests = sum(total_served)
    total_wait_time = sum(total_wait)
    average_wait_time = total_wait_time / total_requests if total_requests != 0 else 0

    print(f"Con {num_servers} servidor(es), el tiempo promedio de espera es: {average_wait_time:.4f} segundos")

    if average_wait_time < 1e-4:  # Aproximado a cero
        print(f"\nSe requieren al menos {num_servers} servidores en Pizzita computing para asegurar que una solicitud nunca espera en cola.")
        break

    num_servers += 1


Con 1 servidor(es), el tiempo promedio de espera es: 1620.0081 segundos
Con 2 servidor(es), el tiempo promedio de espera es: 1440.0367 segundos
Con 3 servidor(es), el tiempo promedio de espera es: 1258.9771 segundos
Con 4 servidor(es), el tiempo promedio de espera es: 1077.1893 segundos
Con 5 servidor(es), el tiempo promedio de espera es: 900.0966 segundos
Con 6 servidor(es), el tiempo promedio de espera es: 720.5146 segundos
Con 7 servidor(es), el tiempo promedio de espera es: 540.2350 segundos
Con 8 servidor(es), el tiempo promedio de espera es: 366.3134 segundos
Con 9 servidor(es), el tiempo promedio de espera es: 179.7534 segundos
Con 10 servidor(es), el tiempo promedio de espera es: 2.9225 segundos
Con 11 servidor(es), el tiempo promedio de espera es: 0.0360 segundos
Con 12 servidor(es), el tiempo promedio de espera es: 0.0121 segundos
Con 13 servidor(es), el tiempo promedio de espera es: 0.0054 segundos
Con 14 servidor(es), el tiempo promedio de espera es: 0.0026 segundos
Con 15 

---------

4. Emita una recomendación para la junta directiva


Después de ejecutar simulaciones con la carga de trabajo esperada a partir del tercer año del lanzamiento de la aplicación (6000 solicitudes por minuto), se encontraron los siguientes resultados:

**Proveedor 1 - Mountain Mega Computing**:
- A pesar de tener un servidor con alta capacidad, no es suficiente para manejar la nueva carga de trabajo. Las solicitudes están esperando en cola un promedio de más de 4 segundos, lo que es bastante alto.
- Aunque puede manejar un gran volumen de solicitudes, el sistema no es escalable.

**Proveedor 2 - Pizzita Computing**:
- Con 10 servidores (que es el número equivalente de capacidad al servidor de Mountain Mega Computing), todavía hay un tiempo de espera en cola, aunque es menor que el Proveedor 1.
- Se determinó que para asegurar que una solicitud nunca tenga que esperar en cola, se necesitarían alquilar 19 servidores de Pizzita Computing. Esto indica escalabilidad en la infraestructura de Pizzita.

**Recomendación**:

A corto plazo (primeros dos años), el Proveedor 1 parece ser una opción más económica y eficiente. Sin embargo, con la expectativa de crecimiento en el tercer año, el Proveedor 1 no podrá manejar el volumen de tráfico sin tiempos de espera significativos.

Por otro lado, Pizzita Computing, aunque inicialmente puede requerir una inversión en más servidores, ofrece escalabilidad. A medida que aumenta el tráfico, simplemente podemos añadir más servidores.

Recomiendo que para los primeros dos años, si la diferencia de costos no es significativa, optemos por Pizzita Computing desde el principio. De esta manera, nos preparamos para el alza esperada en el tercer año y más allá, simplemente añadiendo más servidores a medida que se necesiten. Esto proporcionará una transición sin problemas, evitando cambios de infraestructura en el futuro y asegurando tiempos de respuesta rápidos para los usuarios.
